# GitHub PR Analysis - Jeremy Chia
## Comprehensive Analysis of 1,346 Pull Requests

This notebook provides a deep dive into GitHub contribution patterns, productivity metrics, and collaboration insights based on enhanced PR data from August 2023 to February 2026.

### 📊 Dataset Overview
- **Total PRs**: 1,346 pull requests
- **Time Period**: August 2023 - February 2026  
- **Enhanced Fields**: Descriptions, reviews, comments, reactions, timelines, code metrics
- **Data Source**: GitHub CLI with comprehensive API data extraction

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Libraries imported successfully!")

In [ ]:
# Load the enhanced PR data
df = pd.read_csv('../analysis/enhanced/enhanced_pr_data_complete.csv')

print(f"📊 Dataset loaded successfully!")
print(f"📈 Total PRs: {len(df):,}")
print(f"📋 Columns: {len(df.columns)}")
print(f"📅 Date range: {df['created_at'].min()} to {df['created_at'].max()}")

# Display basic info
df.info()

In [ ]:
# Data preprocessing and cleaning
def clean_and_prepare_data(df):
    """Clean and prepare the DataFrame for analysis"""
    
    # Convert date columns
    date_columns = ['created_at', 'merged_at', 'closed_at']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Extract date components
    df['created_date'] = df['created_at'].dt.date
    df['created_year_month'] = df['created_at'].dt.to_period('M')
    df['created_weekday'] = df['created_at'].dt.day_name()
    df['created_hour'] = df['created_at'].dt.hour
    
    # Clean repository names (extract just the repo name)
    df['repo_short'] = df['repository'].str.split('/').str[-1]
    
    # Create PR size categories
    df['lines_total'] = df['additions'] + df['deletions']
    df['pr_size_category'] = pd.cut(df['lines_total'], 
                                   bins=[0, 10, 50, 200, 1000, float('inf')],
                                   labels=['XS (<10)', 'S (10-50)', 'M (50-200)', 'L (200-1000)', 'XL (1000+)'])
    
    # Create review categories
    df['review_efficiency'] = df['approvals_count'] / (df['reviews_count'] + 1)
    
    # Fill NaN values
    df['time_to_merge_hours'] = df['time_to_merge_hours'].fillna(0)
    df['description_length'] = df['description_length'].fillna(0)
    
    return df

df = clean_and_prepare_data(df)
print("✅ Data cleaned and prepared for analysis!")

# Display sample of cleaned data
df[['repository', 'title', 'state', 'created_date', 'pr_size_category', 'reviews_count']].head()

## 📈 1. High-Level Overview & Key Metrics

Let's start with the big picture of your GitHub contributions.

In [ ]:
# High-level statistics
def display_key_metrics(df):
    """Display key metrics and overview statistics"""
    
    total_prs = len(df)
    merged_prs = len(df[df['state'] == 'MERGED'])
    open_prs = len(df[df['state'] == 'OPEN'])
    
    total_additions = df['additions'].sum()
    total_deletions = df['deletions'].sum()
    total_files = df['files_changed'].sum()
    
    avg_time_to_merge = df[df['time_to_merge_hours'] > 0]['time_to_merge_hours'].mean()
    
    print("🎯 KEY METRICS SUMMARY")
    print("=" * 50)
    print(f"📊 Total PRs Created: {total_prs:,}")
    print(f"✅ Merged PRs: {merged_prs:,} ({merged_prs/total_prs*100:.1f}%)")
    print(f"🔄 Open PRs: {open_prs:,} ({open_prs/total_prs*100:.1f}%)")
    print(f"➕ Total Lines Added: {total_additions:,}")
    print(f"➖ Total Lines Deleted: {total_deletions:,}")
    print(f"📁 Total Files Changed: {total_files:,}")
    print(f"⏱️ Average Time to Merge: {avg_time_to_merge:.1f} hours")
    print(f"🗓️ Analysis Period: {(df['created_at'].max() - df['created_at'].min()).days} days")
    print(f"📅 PRs per Month (avg): {total_prs / ((df['created_at'].max() - df['created_at'].min()).days / 30.44):.1f}")

display_key_metrics(df)

In [ ]:
# Create overview visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('PR Status Distribution', 'PR Size Distribution', 
                   'Monthly PR Creation', 'Repository Distribution (Top 10)'),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# PR Status Distribution
status_counts = df['state'].value_counts()
fig.add_trace(go.Pie(labels=status_counts.index, values=status_counts.values, name="Status"),
              row=1, col=1)

# PR Size Distribution
size_counts = df['pr_size_category'].value_counts()
fig.add_trace(go.Bar(x=size_counts.index, y=size_counts.values, name="Size"),
              row=1, col=2)

# Monthly PR Creation
monthly_prs = df.groupby('created_year_month').size()
fig.add_trace(go.Scatter(x=monthly_prs.index.astype(str), y=monthly_prs.values, 
                        mode='lines+markers', name="Monthly PRs"),
              row=2, col=1)

# Top 10 Repositories
top_repos = df['repo_short'].value_counts().head(10)
fig.add_trace(go.Bar(x=top_repos.values, y=top_repos.index, orientation='h', name="Top Repos"),
              row=2, col=2)

fig.update_layout(height=800, title_text="GitHub PR Analysis - Overview Dashboard", showlegend=False)
fig.show()

## 🚀 2. Productivity Analysis

Understanding your contribution patterns and productivity metrics.

In [ ]:
# Productivity over time analysis
def analyze_productivity_trends(df):
    """Analyze productivity trends over time"""
    
    # Monthly productivity metrics
    monthly_stats = df.groupby('created_year_month').agg({
        'pr_number': 'count',
        'additions': 'sum',
        'deletions': 'sum',
        'files_changed': 'sum',
        'time_to_merge_hours': 'mean'
    }).round(2)
    
    monthly_stats.columns = ['PRs_Created', 'Lines_Added', 'Lines_Deleted', 'Files_Changed', 'Avg_Merge_Time']
    
    return monthly_stats

monthly_productivity = analyze_productivity_trends(df)
print("📊 Monthly Productivity Summary:")
print(monthly_productivity.tail(10))  # Show last 10 months

# Visualize productivity trends
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Monthly PR Creation', 'Monthly Code Changes', 'Average Merge Time'),
    shared_xaxes=True
)

months = monthly_productivity.index.astype(str)

# PRs per month
fig.add_trace(go.Scatter(x=months, y=monthly_productivity['PRs_Created'],
                        mode='lines+markers', name='PRs Created', line=dict(color='blue')),
              row=1, col=1)

# Code changes
fig.add_trace(go.Scatter(x=months, y=monthly_productivity['Lines_Added'],
                        mode='lines+markers', name='Lines Added', line=dict(color='green')),
              row=2, col=1)
fig.add_trace(go.Scatter(x=months, y=monthly_productivity['Lines_Deleted'],
                        mode='lines+markers', name='Lines Deleted', line=dict(color='red')),
              row=2, col=1)

# Merge time
fig.add_trace(go.Scatter(x=months, y=monthly_productivity['Avg_Merge_Time'],
                        mode='lines+markers', name='Avg Merge Time (hours)', line=dict(color='orange')),
              row=3, col=1)

fig.update_layout(height=900, title_text="Productivity Trends Over Time")
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
# Weekly and daily patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Day of week patterns
weekday_counts = df['created_weekday'].value_counts()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts = weekday_counts.reindex(weekday_order)

axes[0,0].bar(weekday_counts.index, weekday_counts.values, color='skyblue')
axes[0,0].set_title('PRs by Day of Week')
axes[0,0].tick_params(axis='x', rotation=45)

# Hour of day patterns
hourly_counts = df['created_hour'].value_counts().sort_index()
axes[0,1].plot(hourly_counts.index, hourly_counts.values, marker='o', color='green')
axes[0,1].set_title('PRs by Hour of Day')
axes[0,1].set_xlabel('Hour')
axes[0,1].set_ylabel('Number of PRs')

# PR size distribution by repository (top 5)
top5_repos = df['repo_short'].value_counts().head(5).index
df_top5 = df[df['repo_short'].isin(top5_repos)]
repo_size_data = []
for repo in top5_repos:
    repo_data = df_top5[df_top5['repo_short'] == repo]['lines_total'].values
    repo_size_data.append(repo_data)

axes[1,0].boxplot(repo_size_data, labels=top5_repos)
axes[1,0].set_title('PR Size Distribution by Repository (Top 5)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].set_ylabel('Total Lines Changed')

# Merge time distribution
merge_times = df[df['time_to_merge_hours'] > 0]['time_to_merge_hours']
axes[1,1].hist(merge_times, bins=30, alpha=0.7, color='orange')
axes[1,1].set_title('Distribution of Merge Times')
axes[1,1].set_xlabel('Hours to Merge')
axes[1,1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"\n🕐 Most active day: {weekday_counts.idxmax()} ({weekday_counts.max()} PRs)")
print(f"🕐 Most active hour: {hourly_counts.idxmax()}:00 ({hourly_counts.max()} PRs)")
print(f"⚡ Median merge time: {merge_times.median():.1f} hours")

## 👥 3. Collaboration & Review Analysis

Analyzing review patterns, approvals, and collaboration metrics.

In [ ]:
# Review and collaboration analysis
def analyze_collaboration(df):
    """Analyze review and collaboration patterns"""
    
    # Review statistics
    total_reviews = df['reviews_count'].sum()
    total_approvals = df['approvals_count'].sum()
    total_change_requests = df['changes_requested_count'].sum()
    
    print("👥 COLLABORATION METRICS")
    print("=" * 40)
    print(f"📝 Total Reviews: {total_reviews:,}")
    print(f"✅ Total Approvals: {total_approvals:,}")
    print(f"🔄 Change Requests: {total_change_requests:,}")
    print(f"📊 Approval Rate: {total_approvals/total_reviews*100:.1f}%" if total_reviews > 0 else "📊 Approval Rate: N/A")
    
    # PRs with different review counts
    review_distribution = df['reviews_count'].value_counts().sort_index()
    print(f"\n📈 Review Distribution:")
    for reviews, count in review_distribution.head(6).items():
        print(f"  {reviews} reviews: {count} PRs ({count/len(df)*100:.1f}%)")
    
    return {
        'total_reviews': total_reviews,
        'approval_rate': total_approvals/total_reviews if total_reviews > 0 else 0
    }

collab_stats = analyze_collaboration(df)

In [ ]:
# Visualize collaboration patterns
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Review Count Distribution', 'Approval vs Change Requests', 
                   'Comments vs Reviews Correlation', 'Review Efficiency by Repository'),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "box"}]]
)

# Review count distribution
review_dist = df['reviews_count'].value_counts().sort_index()
fig.add_trace(go.Bar(x=review_dist.index[:10], y=review_dist.values[:10], 
                    name="Review Distribution"),
              row=1, col=1)

# Approval vs Change Requests scatter
fig.add_trace(go.Scatter(x=df['approvals_count'], y=df['changes_requested_count'],
                        mode='markers', name="Approval vs Changes",
                        text=df['title'].str[:30], hovertemplate='%{text}<br>Approvals: %{x}<br>Changes: %{y}'),
              row=1, col=2)

# Comments vs Reviews correlation
fig.add_trace(go.Scatter(x=df['reviews_count'], y=df['comments_count'],
                        mode='markers', name="Comments vs Reviews",
                        text=df['title'].str[:30]),
              row=2, col=1)

# Review efficiency by top repositories
top_repos_for_reviews = df['repo_short'].value_counts().head(5).index
for i, repo in enumerate(top_repos_for_reviews):
    repo_data = df[df['repo_short'] == repo]
    efficiency = repo_data['review_efficiency']
    fig.add_trace(go.Box(y=efficiency, name=repo, showlegend=False),
                  row=2, col=2)

fig.update_layout(height=800, title_text="Collaboration & Review Analysis")
fig.show()

In [ ]:
# Most collaborative repositories and reviewers analysis
print("🤝 MOST COLLABORATIVE REPOSITORIES:")
print("=" * 45)

repo_collab = df.groupby('repo_short').agg({
    'reviews_count': 'mean',
    'approvals_count': 'mean',
    'comments_count': 'mean',
    'pr_number': 'count'
}).round(2)

repo_collab = repo_collab[repo_collab['pr_number'] >= 5]  # Only repos with 5+ PRs
repo_collab = repo_collab.sort_values('reviews_count', ascending=False)

print(repo_collab.head(10))

# Analyze unique reviewers (extract from the unique_reviewers field)
print("\n👨‍💼 REVIEWER ANALYSIS:")
print("=" * 30)

# Count total unique reviewer mentions across all PRs
all_reviewers = []
for reviewers_str in df['unique_reviewers'].dropna():
    if reviewers_str and reviewers_str != '':
        reviewers = reviewers_str.split('|')
        all_reviewers.extend([r.strip() for r in reviewers if r.strip()])

if all_reviewers:
    reviewer_counts = pd.Series(all_reviewers).value_counts()
    print("Top 10 Most Frequent Reviewers:")
    print(reviewer_counts.head(10))
else:
    print("No reviewer data available in the dataset.")

## 📊 4. Repository Specialization Analysis

Understanding your contribution patterns across different repositories.

In [ ]:
# Repository analysis
def analyze_repositories(df):
    """Comprehensive repository analysis"""
    
    repo_stats = df.groupby('repo_short').agg({
        'pr_number': 'count',
        'additions': ['sum', 'mean'],
        'deletions': ['sum', 'mean'], 
        'files_changed': ['sum', 'mean'],
        'time_to_merge_hours': 'mean',
        'reviews_count': 'mean',
        'state': lambda x: (x == 'MERGED').sum() / len(x)
    }).round(2)
    
    # Flatten column names
    repo_stats.columns = ['Total_PRs', 'Total_Additions', 'Avg_Additions',
                         'Total_Deletions', 'Avg_Deletions', 'Total_Files', 'Avg_Files',
                         'Avg_Merge_Time', 'Avg_Reviews', 'Merge_Rate']
    
    repo_stats = repo_stats.sort_values('Total_PRs', ascending=False)
    return repo_stats

repo_analysis = analyze_repositories(df)
print("🏆 TOP REPOSITORIES BY CONTRIBUTION:")
print("=" * 50)
print(repo_analysis.head(15))

In [ ]:
# Repository specialization visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Repository Contribution Distribution', 'Avg PR Size by Repository', 
                   'Merge Rate by Repository', 'Repository Activity Heatmap')
)

# Top 15 repositories by PR count
top_repos = repo_analysis.head(15)

# Contribution distribution (treemap style as bar chart)
fig.add_trace(go.Bar(x=top_repos.index, y=top_repos['Total_PRs'],
                    name="PR Count", marker_color='lightblue'),
              row=1, col=1)

# Average PR size by repository  
fig.add_trace(go.Scatter(x=top_repos['Avg_Additions'], y=top_repos['Avg_Deletions'],
                        mode='markers+text', text=top_repos.index,
                        textposition="top center", name="Avg Size",
                        marker=dict(size=top_repos['Total_PRs']/2, opacity=0.6)),
              row=1, col=2)

# Merge rate by repository
fig.add_trace(go.Bar(x=top_repos.index, y=top_repos['Merge_Rate'],
                    name="Merge Rate", marker_color='lightgreen'),
              row=2, col=1)

# Monthly activity heatmap for top 5 repos
top_5_repos = df['repo_short'].value_counts().head(5).index
heatmap_data = []
months = sorted(df['created_year_month'].unique())

for repo in top_5_repos:
    repo_monthly = df[df['repo_short'] == repo].groupby('created_year_month').size()
    repo_row = [repo_monthly.get(month, 0) for month in months]
    heatmap_data.append(repo_row)

fig.add_trace(go.Heatmap(z=heatmap_data, 
                        x=[str(m) for m in months][-12:],  # Last 12 months
                        y=list(top_5_repos),
                        colorscale='Blues', name="Activity"),
              row=2, col=2)

fig.update_layout(height=800, title_text="Repository Specialization Analysis")
fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=2, col=1)
fig.update_xaxes(tickangle=45, row=2, col=2)
fig.show()

In [ ]:
# Repository specialization insights
print("🎯 REPOSITORY SPECIALIZATION INSIGHTS:")
print("=" * 45)

# Identify repository types based on patterns
repo_analysis['Impact_Score'] = (repo_analysis['Total_Additions'] + repo_analysis['Total_Deletions']) / repo_analysis['Total_PRs']
repo_analysis['Collaboration_Score'] = repo_analysis['Avg_Reviews']

# Categorize repositories
high_volume = repo_analysis[repo_analysis['Total_PRs'] >= 50]
high_impact = repo_analysis[repo_analysis['Impact_Score'] >= 100]
collaborative = repo_analysis[repo_analysis['Avg_Reviews'] >= 1.5]

print(f"📈 High Volume Repos (50+ PRs): {len(high_volume)}")
if not high_volume.empty:
    print(f"   Top: {high_volume.index[0]} ({high_volume.iloc[0]['Total_PRs']:.0f} PRs)")

print(f"💥 High Impact Repos (100+ lines/PR): {len(high_impact)}")
if not high_impact.empty:
    print(f"   Top: {high_impact.sort_values('Impact_Score', ascending=False).index[0]} ({high_impact.sort_values('Impact_Score', ascending=False).iloc[0]['Impact_Score']:.0f} lines/PR)")

print(f"👥 Collaborative Repos (1.5+ reviews/PR): {len(collaborative)}")
if not collaborative.empty:
    print(f"   Top: {collaborative.sort_values('Avg_Reviews', ascending=False).index[0]} ({collaborative.sort_values('Avg_Reviews', ascending=False).iloc[0]['Avg_Reviews']:.1f} reviews/PR)")

# Show repository diversity
total_repos = len(repo_analysis)
print(f"\n🏗️ Total Repositories Contributed To: {total_repos}")
print(f"📊 Repository Diversity Index: {1 - (repo_analysis['Total_PRs']**2).sum() / (repo_analysis['Total_PRs'].sum()**2):.3f}")
print("   (Higher = more evenly distributed across repos)")

## ⏱️ 5. Timeline & Efficiency Analysis

Understanding merge times, workflow efficiency, and temporal patterns.

In [ ]:
# Timeline and efficiency analysis
def analyze_timeline_efficiency(df):
    """Analyze timeline patterns and workflow efficiency"""
    
    # Filter for merged PRs with valid merge times
    merged_df = df[(df['state'] == 'MERGED') & (df['time_to_merge_hours'] > 0)]
    
    if len(merged_df) == 0:
        print("❌ No merged PRs with valid merge times found.")
        return
    
    print("⏱️ TIMELINE & EFFICIENCY METRICS")
    print("=" * 40)
    print(f"📊 Analyzed Merged PRs: {len(merged_df):,}")
    print(f"⚡ Median Merge Time: {merged_df['time_to_merge_hours'].median():.1f} hours")
    print(f"📈 Average Merge Time: {merged_df['time_to_merge_hours'].mean():.1f} hours")
    print(f"🚀 Fastest Merge: {merged_df['time_to_merge_hours'].min():.1f} hours")
    print(f"🐌 Slowest Merge: {merged_df['time_to_merge_hours'].max():.1f} hours")
    
    # Efficiency categories
    fast_merges = len(merged_df[merged_df['time_to_merge_hours'] <= 24])  # Same day
    medium_merges = len(merged_df[(merged_df['time_to_merge_hours'] > 24) & 
                                 (merged_df['time_to_merge_hours'] <= 168)])  # Within a week
    slow_merges = len(merged_df[merged_df['time_to_merge_hours'] > 168])  # Over a week
    
    print(f"\n🚀 Fast Merges (≤24h): {fast_merges} ({fast_merges/len(merged_df)*100:.1f}%)")
    print(f"⚡ Medium Merges (1-7 days): {medium_merges} ({medium_merges/len(merged_df)*100:.1f}%)")
    print(f"🐌 Slow Merges (>7 days): {slow_merges} ({slow_merges/len(merged_df)*100:.1f}%)")
    
    return merged_df

merged_prs = analyze_timeline_efficiency(df)

if merged_prs is not None and not merged_prs.empty:
    # Merge time analysis by various factors
    print("\n📊 MERGE TIME BY FACTORS:")
    print("=" * 35)
    
    # By PR size
    size_merge_times = merged_prs.groupby('pr_size_category')['time_to_merge_hours'].median().sort_values()
    print("By PR Size (median hours):")
    for size, time in size_merge_times.items():
        print(f"  {size}: {time:.1f}h")
    
    # By repository (top 5)
    top_repos_merge = merged_prs['repo_short'].value_counts().head(5).index
    repo_merge_times = merged_prs[merged_prs['repo_short'].isin(top_repos_merge)].groupby('repo_short')['time_to_merge_hours'].median().sort_values()
    print(f"\nBy Repository (top 5, median hours):")
    for repo, time in repo_merge_times.items():
        print(f"  {repo}: {time:.1f}h")

In [ ]:
# Timeline visualization
if merged_prs is not None and not merged_prs.empty:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Merge Time Distribution', 'Merge Time vs PR Size', 
                       'Merge Time Trends Over Time', 'Merge Efficiency by Day of Week')
    )
    
    # Merge time distribution (log scale for better visualization)
    merge_times_hours = merged_prs['time_to_merge_hours']
    fig.add_trace(go.Histogram(x=np.log10(merge_times_hours + 1), 
                              name="Log10(Hours+1)", nbinsx=30),
                  row=1, col=1)
    
    # Merge time vs PR size
    for size_cat in merged_prs['pr_size_category'].dropna().unique():
        size_data = merged_prs[merged_prs['pr_size_category'] == size_cat]['time_to_merge_hours']
        fig.add_trace(go.Box(y=size_data, name=str(size_cat), showlegend=False),
                      row=1, col=2)
    
    # Merge time trends over time (monthly averages)
    monthly_merge_times = merged_prs.groupby('created_year_month')['time_to_merge_hours'].mean()
    fig.add_trace(go.Scatter(x=monthly_merge_times.index.astype(str), 
                            y=monthly_merge_times.values,
                            mode='lines+markers', name="Monthly Avg",
                            line=dict(color='red')),
                  row=2, col=1)
    
    # Merge efficiency by day of week
    weekday_merge_times = merged_prs.groupby('created_weekday')['time_to_merge_hours'].median()
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday_merge_times = weekday_merge_times.reindex(weekday_order)
    
    fig.add_trace(go.Bar(x=weekday_merge_times.index, y=weekday_merge_times.values,
                        name="Median Merge Time", marker_color='lightcoral'),
                  row=2, col=2)
    
    fig.update_layout(height=800, title_text="Timeline & Efficiency Analysis")
    fig.show()
else:
    print("⚠️ Cannot create timeline visualizations - no valid merge time data.")

## 🎯 6. Key Insights & Recommendations

Summary of findings and actionable insights from the analysis.

In [ ]:
# Generate key insights and recommendations
def generate_insights(df, repo_analysis, merged_prs):
    """Generate key insights and recommendations based on the analysis"""
    
    insights = []
    
    # Productivity insights
    total_prs = len(df)
    avg_monthly_prs = total_prs / ((df['created_at'].max() - df['created_at'].min()).days / 30.44)
    top_repo = df['repo_short'].value_counts().index[0]
    top_repo_count = df['repo_short'].value_counts().iloc[0]
    
    insights.append(f"🚀 **High Productivity**: You've created {total_prs:,} PRs with an average of {avg_monthly_prs:.1f} PRs per month.")
    insights.append(f"🎯 **Repository Focus**: Your top repository '{top_repo}' accounts for {top_repo_count} PRs ({top_repo_count/total_prs*100:.1f}% of total contributions).")
    
    # Code impact insights
    total_lines = df['additions'].sum() + df['deletions'].sum()
    avg_pr_size = total_lines / total_prs
    large_prs = len(df[df['lines_total'] > 200])
    
    insights.append(f"💻 **Code Impact**: You've changed {total_lines:,} lines across all PRs (avg: {avg_pr_size:.0f} lines per PR).")
    insights.append(f"📏 **PR Sizing**: {large_prs} PRs are large (200+ lines), representing {large_prs/total_prs*100:.1f}% of your work.")
    
    # Collaboration insights
    reviewed_prs = len(df[df['reviews_count'] > 0])
    avg_reviews = df[df['reviews_count'] > 0]['reviews_count'].mean() if reviewed_prs > 0 else 0
    
    insights.append(f"👥 **Collaboration**: {reviewed_prs} PRs received reviews ({reviewed_prs/total_prs*100:.1f}%), with an average of {avg_reviews:.1f} reviews per reviewed PR.")
    
    # Efficiency insights
    if merged_prs is not None and not merged_prs.empty:
        fast_merge_rate = len(merged_prs[merged_prs['time_to_merge_hours'] <= 24]) / len(merged_prs) * 100
        median_merge = merged_prs['time_to_merge_hours'].median()
        
        insights.append(f"⚡ **Merge Efficiency**: {fast_merge_rate:.1f}% of PRs merge within 24 hours, with a median merge time of {median_merge:.1f} hours.")
    
    # Repository diversity
    repo_count = len(df['repo_short'].unique())
    insights.append(f"🏗️ **Repository Diversity**: You contribute to {repo_count} different repositories, showing broad technical engagement.")
    
    # Timing insights
    most_active_day = df['created_weekday'].value_counts().index[0]
    most_active_hour = df['created_hour'].value_counts().index[0]
    insights.append(f"📅 **Activity Patterns**: Most active on {most_active_day}s around {most_active_hour}:00, indicating consistent work patterns.")
    
    return insights

# Generate and display insights
key_insights = generate_insights(df, repo_analysis, merged_prs)

print("🎯 KEY INSIGHTS & FINDINGS")
print("=" * 50)
for i, insight in enumerate(key_insights, 1):
    print(f"{i}. {insight}")

print("\n💡 RECOMMENDATIONS")
print("=" * 30)
print("1. 🎯 **Focus Areas**: Consider consolidating efforts on fewer repositories for deeper impact")
print("2. 📏 **PR Sizing**: Break down large PRs (200+ lines) for faster review cycles")  
print("3. ⚡ **Review Speed**: Leverage your fast merge patterns to establish best practices")
print("4. 👥 **Collaboration**: Maintain high review engagement to ensure code quality")
print("5. 📊 **Tracking**: Use these metrics as KPIs for ongoing productivity measurement")

In [ ]:
# Create a final summary dashboard
summary_metrics = {
    'Total PRs': len(df),
    'Merged Rate': f"{len(df[df['state'] == 'MERGED'])/len(df)*100:.1f}%",
    'Avg PR Size': f"{(df['additions'] + df['deletions']).mean():.0f} lines",
    'Top Repository': df['repo_short'].value_counts().index[0],
    'Total Repositories': len(df['repo_short'].unique()),
    'Avg Reviews per PR': f"{df['reviews_count'].mean():.1f}",
    'Lines of Code Changed': f"{(df['additions'] + df['deletions']).sum():,}",
    'Most Active Day': df['created_weekday'].value_counts().index[0]
}

print("📊 FINAL SUMMARY DASHBOARD")
print("=" * 50)
for metric, value in summary_metrics.items():
    print(f"📈 {metric}: {value}")

print(f"\n🎉 Analysis Complete! This comprehensive analysis covers {len(df):,} PRs across {len(df['repo_short'].unique())} repositories.")
print("💾 All visualizations and metrics are available for further exploration and sharing.")

## 📝 Next Steps

This analysis provides a comprehensive overview of your GitHub contributions. You can:

1. **Deep Dive**: Focus on specific repositories or time periods for detailed analysis
2. **Comparison**: Compare your metrics against team averages or industry benchmarks  
3. **Optimization**: Use insights to optimize your workflow and collaboration patterns
4. **Reporting**: Export key visualizations for presentations or performance reviews
5. **Monitoring**: Set up regular analysis to track changes in your contribution patterns

### 🔄 Refreshing the Analysis
To update this analysis with new data:
1. Re-run the PR enhancement script: `cd analysis/tools && python3 enhance_pr_data.py`
2. Restart this notebook and run all cells with the updated data

---
*Analysis generated on February 21, 2026 | Data source: GitHub CLI enhanced extraction*